In [1]:
import pandas as pd
import numpy as np
import json
import requests
import datetime
import twint
import nest_asyncio
nest_asyncio.apply()
import config_v2 as cfg
pd.set_option('display.float_format', lambda x: '%.3f' % x) #avoid scientific notation
import time

In [2]:
# monthly
# fetching btc price with open, low, high, close for 30 days, provides 4hr window
base_url = 'https://api.coingecko.com/api/v3'
url = base_url+'/coins/bitcoin/ohlc?vs_currency=usd&days=30'
response = requests.get(url)
btc_ohlc_30 = response.json()
btc_ohlc_30_df = pd.DataFrame(btc_ohlc_30, columns=('timestamp', 'open', 'high', 'low', 'close'))
btc_ohlc_30_df['datetime'] = btc_ohlc_30_df.timestamp.apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
btc_ohlc_30_df

old_btc_ohlc_30_df = pd.read_csv('btc_ohlc_30.csv')
btc_ohlc_30_df = btc_ohlc_30_df.append(old_btc_ohlc_30_df).reset_index(drop=True).drop_duplicates()
btc_ohlc_30_df.to_csv('btc_ohlc_30.csv',index=False)
print('old_btc_ohlc_30_df.shape: \t', old_btc_ohlc_30_df.shape)
print('btc_ohlc_30_df.shape: \t\t', old_btc_ohlc_30_df.shape)
print('Saved successfully. \nAdded: {}.'.format(len(btc_ohlc_30_df)-len(old_btc_ohlc_30_df)))

old_btc_ohlc_30_df.shape: 	 (364, 6)
btc_ohlc_30_df.shape: 		 (364, 6)
Saved successfully. 
Added: 0.


In [36]:
# monthly/weekly
# scrapping tweets from 'whale_alert'
date_from_ls = []
date_until_ls = []
n_weeks = 2 #10 #22

for week in range(n_weeks):
#    print(datetime.datetime.today().date() - datetime.timedelta(days=7*week))
    yesterday = datetime.datetime.today().date() - datetime.timedelta(days=1)
    date_from = yesterday - datetime.timedelta(days=7*(week+1))
    date_until = yesterday - datetime.timedelta(days=7*(week))
    date_from_ls.append(str(date_from)) 
    date_until_ls.append(str(date_until))
#    print('\tfrom: {} \t until: {}'.format(date_from,date_until))
print('date_from: ', date_from_ls)
print('date_until: ', date_until_ls)
dates_np = np.array([date_from_ls,date_until_ls])

tw_whale_df = pd.DataFrame()
n_weeks = dates_np.shape[1]
for i in range(n_weeks):
        
    c = twint.Config()
    c.Username = 'whale_alert'
    c.Pandas = True
    #c.Lang = 'en'
    #c.Limit = 200
    #c.Count = True
    c.Since = dates_np[0,i]
    c.Until = dates_np[1,i]
    twint.run.Search(c)
    tw_whale_df = tw_whale_df.append(twint.storage.panda.Tweets_df)
    time.sleep(3)

old_tw_whale_df = pd.read_csv('tw_whale.csv')
print('old_tw_whale_df.shape: \t', old_tw_whale_df.shape)
tw_whale_df = tw_whale_df.append(old_tw_whale_df).reset_index(drop=True)
tw_whale_df['id'] = tw_whale_df.id.astype('int64')
tw_whale_df.drop_duplicates(subset='id', inplace=True)
print('tw_whale_df.shape: \t', tw_whale_df.shape)
tw_whale_df.to_csv('tw_whale.csv', index=False)
print('Saved successfully. \nAdded: {}.'.format(len(tw_whale_df)-len(old_tw_whale_df)))

date_from:  ['2021-05-23', '2021-05-16']
date_until:  ['2021-05-30', '2021-05-23']
1398780698576699393 2021-05-30 01:18:01 +0200 <whale_alert> 🔥 🔥  13,600,000 #USDC (13,600,000 USD) burned at USDC Treasury   https://t.co/Tng77tTf90
1398780176541044737 2021-05-30 01:15:56 +0200 <whale_alert> 🔥 🔥  10,973,421 #USDC (10,973,421 USD) burned at USDC Treasury   https://t.co/d27mdNlzJn
1398778483791257604 2021-05-30 01:09:13 +0200 <whale_alert> 🚨 🚨 🚨  30,033,040 #USDT (30,033,040 USD) transferred from #FTX to unknown wallet   https://t.co/jFvparuqhp
1398777995549200385 2021-05-30 01:07:16 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨  50,165,001 #USDC (50,165,001 USD) transferred from #Huobi to #FTX   https://t.co/wZvPqGYPiU
1398777276410499072 2021-05-30 01:04:25 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨  50,165,001 #USDC (50,165,001 USD) transferred from unknown wallet to #Huobi   https://t.co/dfopXt4LCQ
1398777012370739203 2021-05-30 01:03:22 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨  50,075,000 #USDC (50,075,000 USD) transferred

1398432096729829381 2021-05-29 02:12:48 +0200 <whale_alert> 🚨  2,000,000 #EOS (12,372,877 USD) transferred from #Bybit to unknown wallet   https://t.co/zzgtNmv2Gx
1398429134934708226 2021-05-29 02:01:01 +0200 <whale_alert> 💵  8,969,015 #USDC (8,969,015 USD) minted at USDC Treasury   https://t.co/PF8orlKGrO
1398424773152096256 2021-05-29 01:43:42 +0200 <whale_alert> 🔥  5,678,909 #HUSD (5,678,909 USD) burned at HUSD Incinerator   https://t.co/IgaH0n57zc
1398424373145522177 2021-05-29 01:42:06 +0200 <whale_alert> 🔥  7,889,092 #HUSD (7,889,092 USD) burned at HUSD Incinerator   https://t.co/kJphSMTWQL
1398413311922802689 2021-05-29 00:58:09 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  11,523 #BTC (400,704,852 USD) transferred from unknown wallet to unknown wallet   https://t.co/h5yun7vA6r
1398413309867659265 2021-05-29 00:58:08 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  12,023 #BTC (418,091,657 USD) transferred from unknown wallet to unknown wallet   https://t.co/KxBkoHPtRh
1398407987589521411 20

1398103856245096452 2021-05-28 04:28:29 +0200 <whale_alert> 💵 💵  13,633,026 #USDC (13,633,026 USD) minted at USDC Treasury   https://t.co/9W0dXWdXkb
1398091691148652545 2021-05-28 03:40:09 +0200 <whale_alert> 🚨 🚨  24,999,998 #USDC (24,999,998 USD) transferred from #FTX to unknown wallet   https://t.co/lXtZW5J59X
1398084379289731073 2021-05-28 03:11:05 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨  21,312 #ETH (57,461,584 USD) transferred from #Binance to unknown wallet   https://t.co/qM59d9Ad65
1398083369783726080 2021-05-28 03:07:05 +0200 <whale_alert> 🚨 🚨  20,075,998 #USDC (20,075,998 USD) transferred from #Huobi to #FTX   https://t.co/Oc4DqLtDml
1398083127592079362 2021-05-28 03:06:07 +0200 <whale_alert> 🚨 🚨  20,075,998 #USDC (20,075,998 USD) transferred from unknown wallet to #Huobi   https://t.co/TvKmT5pG75
1398081053320003584 2021-05-28 02:57:52 +0200 <whale_alert> 🚨 🚨 🚨  800 #BTC (30,530,856 USD) transferred from unknown wallet to CryptoCom   https://t.co/3Y2FUIIdjq
1398080653862834176 2021-05-

1397826690580107266 2021-05-27 10:07:08 +0200 <whale_alert> 15,000,000 #CHZ (4,693,964 USD) transferred from unknown wallet to #Binance   https://t.co/WLXQMfWNiK
1397826582853689347 2021-05-27 10:06:42 +0200 <whale_alert> 🔥 🔥 🔥 🔥 🔥 🔥 🔥 🔥 🔥 🔥  146,143,564 #BUSD (146,143,564 USD) burned at #Binance   https://t.co/A2swJdCXVw
1397826512812969988 2021-05-27 10:06:25 +0200 <whale_alert> 🚨  19,995,515 #USDC (19,995,515 USD) transferred from USDC Treasury to #Huobi   https://t.co/iDi7txqZWF
1397826509625348097 2021-05-27 10:06:24 +0200 <whale_alert> 🚨  19,995,525 #USDC (19,995,525 USD) transferred from USDC Treasury to #Huobi   https://t.co/lvSHJNYVV8
1397825041115631617 2021-05-27 10:00:34 +0200 <whale_alert> 🚨  19,991,505 #BUSD (19,991,505 USD) transferred from unknown wallet to #Binance   https://t.co/dcd1TG3PNj
1397825037986639872 2021-05-27 10:00:34 +0200 <whale_alert> 💵 💵  11,011,164 #USDC (11,011,164 USD) minted at USDC Treasury   https://t.co/4pMvwZaXFn
1397824636088430594 2021-05-27 0

1397575576819544069 2021-05-26 17:29:17 +0200 <whale_alert> 🚨  3,969 #ETH (10,723,752 USD) transferred from #Bitfinex to unknown wallet   https://t.co/IPfTDp8Z9h
1397569778617815042 2021-05-26 17:06:15 +0200 <whale_alert> 64,999,999 #TRX (5,212,859 USD) transferred from #Binance to unknown wallet   https://t.co/6YbmmcpBh1
1397568157091500041 2021-05-26 16:59:48 +0200 <whale_alert> 🚨 🚨 🚨  816 #BTC (32,046,762 USD) transferred from #Bitstamp to unknown wallet   https://t.co/KySRGLCvXE
1397560287130230791 2021-05-26 16:28:32 +0200 <whale_alert> 1,988,704 #XTZ (7,486,883 USD) transferred from #Kraken to unknown wallet   https://t.co/P5EmSFCSUG
1397554543521632264 2021-05-26 16:05:43 +0200 <whale_alert> 1,263,715 #EOS (7,646,007 USD) transferred from #Okex to unknown wallet   https://t.co/hfJFSCueYZ
1397554203300667399 2021-05-26 16:04:22 +0200 <whale_alert> 🚨 🚨  23,990,740 #USDT (23,990,740 USD) transferred from unknown wallet to #Okex   https://t.co/IdrHMpPAp6
1397547734832488457 2021-05-

...

1394682776583028745 2021-05-18 17:54:20 +0200 <whale_alert> 🚨 🚨  20,000,000 #USDC (20,000,000 USD) transferred from #FTX to unknown wallet   https://t.co/9orRNIeTSj
1394682624090726402 2021-05-18 17:53:44 +0200 <whale_alert> 🔥  153,864 #FTT (7,890,978 USD) burned at #FTX   https://t.co/ciOF97b8pT
1394681996564127750 2021-05-18 17:51:14 +0200 <whale_alert> 🚨 🚨  24,909,925 #USDC (24,909,925 USD) transferred from #Huobi to #FTX   https://t.co/qs9QB2xiQI
1394680578788798464 2021-05-18 17:45:36 +0200 <whale_alert> 💵 💵 💵  23,810,765 #USDC (23,810,765 USD) minted at USDC Treasury   https://t.co/NABpSgPMru
1394677088947429378 2021-05-18 17:31:44 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  107,999,978 #BUSD (107,999,978 USD) transferred from unknown wallet to unknown wallet   https://t.co/IsDACQBjQR
1394676922077097984 2021-05-18 17:31:04 +0200 <whale_alert> 💵 💵 💵 💵  39,472,074 #BUSD (39,472,074 USD) minted at unknown wallet   https://t.co/YETQxhFoC1
1394676918927101952 2021-05-18 17:31:03 +0200 <

1394521353089396739 2021-05-18 07:12:54 +0200 <whale_alert> 🚨 🚨 🚨 🚨  12,000 #ETH (42,243,516 USD) transferred from #Binance to unknown wallet   https://t.co/QuoSTC1NNY
1394503259465846788 2021-05-18 06:01:00 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  123,999,989 #BUSD (123,999,989 USD) transferred from unknown wallet to #Binance   https://t.co/OihP0LDKLv
1394501529420320769 2021-05-18 05:54:07 +0200 <whale_alert> 🔥  7,388,769 #HUSD (7,388,769 USD) burned at HUSD Incinerator   https://t.co/05x234psRZ
1394500525383720970 2021-05-18 05:50:08 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  123,999,989 #BUSD (123,999,989 USD) transferred from unknown wallet to unknown wallet   https://t.co/MkVgkbM6T5
1394500287122001920 2021-05-18 05:49:11 +0200 <whale_alert> 💵 💵 💵 💵 💵 💵 💵 💵 💵 💵  117,127,147 #BUSD (117,127,147 USD) minted at unknown wallet   https://t.co/NshaTEMGO4
1394500284169207808 2021-05-18 05:49:10 +0200 <whale_alert> 💵 💵  18,516,780 #PAX (18,586,338 USD) minted at unknown wallet   https://t.c

1394279183690436611 2021-05-17 15:10:36 +0200 <whale_alert> 🚨 🚨 🚨  30,000,000 #USDT (30,000,000 USD) transferred from Tether Treasury to unknown wallet   https://t.co/YnnI1l2Z12
1394278034811858950 2021-05-17 15:06:02 +0200 <whale_alert> 🚨 🚨 🚨 🚨  41,000,000 #USDT (41,000,000 USD) transferred from #Binance to unknown wallet   https://t.co/e13ex8Tv2b
1394276875342581760 2021-05-17 15:01:26 +0200 <whale_alert> 🚨 🚨  6,000 #ETH (20,820,754 USD) transferred from #Huobi to unknown wallet   https://t.co/m7ViRSO47x
1394275500202373121 2021-05-17 14:55:58 +0200 <whale_alert> 🔥  7,742,316 #USDC (7,742,316 USD) burned at USDC Treasury   https://t.co/j5f5rcA6ls
1394271777614934020 2021-05-17 14:41:10 +0200 <whale_alert> 500,000 #XTZ (2,702,830 USD) transferred from #Kraken to unknown wallet   https://t.co/KHvaBfLlf1
1394269373217574912 2021-05-17 14:31:37 +0200 <whale_alert> 🚨 🚨  24,999,986 #BUSD (24,999,986 USD) transferred from unknown wallet to #Binance   https://t.co/xv7Zv7IKwx
1394267920474202

1394152631275294726 2021-05-17 06:47:44 +0200 <whale_alert> 🔥 🔥 🔥  27,209,124 #USDC (27,209,124 USD) burned at USDC Treasury   https://t.co/MpfQ5ghuiV
1394152546185486339 2021-05-17 06:47:23 +0200 <whale_alert> 🚨 🚨 🚨  39,000,000 #USDT (39,000,000 USD) transferred from unknown wallet to #Binance   https://t.co/EES2Tt3PGY
1394145314832097284 2021-05-17 06:18:39 +0200 <whale_alert> 4,988,584 #GRT (6,087,610 USD) transferred from unknown wallet to #Binance   https://t.co/Se5RvHvaUH
1394144858491789313 2021-05-17 06:16:50 +0200 <whale_alert> 🚨 🚨  20,000,000 #USDT (20,000,000 USD) transferred from #Binance to Tether Treasury   https://t.co/UadZb5Apl7
1394142010785812481 2021-05-17 06:05:31 +0200 <whale_alert> 4,988,584 #GRT (5,995,701 USD) transferred from #FTX to unknown wallet   https://t.co/0Y0xF1ZsyE
1394141956964560902 2021-05-17 06:05:19 +0200 <whale_alert> 4,214,497 #GRT (5,065,338 USD) transferred from #Huobi to #FTX   https://t.co/elpjIs7l5q
1394139908802064385 2021-05-17 05:57:10 +

1393830644548050944 2021-05-16 09:28:16 +0200 <whale_alert> 7,000,000 #XLM (5,256,576 USD) transferred from unknown wallet to #Kraken   https://t.co/y7bVru5KUJ
1393825271694991363 2021-05-16 09:06:55 +0200 <whale_alert> 🚨  3,180 #ETH (12,289,861 USD) transferred from unknown wallet to #Coinone   https://t.co/mKmKgIEPpW
1393817834904502273 2021-05-16 08:37:22 +0200 <whale_alert> 🚨 🚨 🚨  10,000 #ETH (38,695,971 USD) transferred from unknown wallet to #Gemini   https://t.co/uL0rJLT8ts
1393817618570678274 2021-05-16 08:36:30 +0200 <whale_alert> 🚨  231,980 #LINK (10,243,794 USD) transferred from unknown wallet to #Binance   https://t.co/Ajf1Tv9W0y
1393807099231092736 2021-05-16 07:54:42 +0200 <whale_alert> 3,042,330 #MATIC (5,199,905 USD) transferred from unknown wallet to #Binance   https://t.co/cij5y3TrYs
1393799593679335427 2021-05-16 07:24:53 +0200 <whale_alert> 🚨 🚨  600 #WBTC (28,819,815 USD) transferred from WBTC Treasury to unknown wallet   https://t.co/GYK3EjAXYR
1393797884894720000 